In [1]:
import torch
from models.model_partseg import Net
from data import ShapeNetPart_Augmented
from torch.utils.data import DataLoader
import pickle
import os
from tqdm.notebook import tqdm

In [2]:
with open("args.pkl", "rb") as f:
    args = pickle.load(f)

In [3]:
model = Net(args).cuda(4)

In [4]:
ds = ShapeNetPart_Augmented("trainval", True)

In [5]:
dl = DataLoader(ds, batch_size=8)

In [6]:
# nan = 0

# for x,y,z in tqdm(dl):
#     with torch.no_grad():
#         x,y=x.cuda(4),y.cuda(4)
#         x=x.transpose(1,2)
#         y= torch.eye(16)[y].cuda(4).squeeze()
#         r = model(x,y)
#         nan += torch.isnan(r).sum()

In [7]:
from loss import cross_entropy
from torch.optim import AdamW

In [8]:
opt = AdamW(model.parameters(), lr=2e-3, weight_decay=0.1)

In [9]:
total_loss = 0
for x, y, z in tqdm(dl):
    opt.zero_grad()
    x, y, z = x.cuda(4), y.cuda(4), z.cuda(4)
    x = x.transpose(1, 2)
    y = torch.eye(16)[y].cuda(4).squeeze()
    r = model(x, y).transpose(1,2).contiguous()
    loss = cross_entropy(r.view(-1, 50),z)
    total_loss += loss.item()
    if torch.isnan(loss).item():
        print("NAN Loss!!!!")
    loss.backward()
    opt.step()

  0%|          | 0/1751 [00:00<?, ?it/s]

In [10]:
total_loss

3600.3940851688385

In [11]:
loss

tensor(2.1468, device='cuda:4', grad_fn=<NegBackward0>)